# EdgeLLM CUDA vs Ollama Benchmark

Comprehensive throughput comparison between EdgeLLM CUDA kernels and Ollama.

**Goal**: Demonstrate higher tok/s with our CUDA T-MAC kernels.

**Hardware**: Tesla T4 (15GB VRAM, 40 SMs, Compute 7.5)

## 1. Environment Setup

In [ ]:
# Check GPU
!nvidia-smi --query-gpu=name,memory.total,compute_cap --format=csv

In [ ]:
# Clone and build
import os
if not os.path.exists('ollama-api-gateway'):
    !git clone https://github.com/umerkhan95/ollama-api-gateway.git
else:
    !cd ollama-api-gateway && git pull

%cd ollama-api-gateway/mojo-gateway/src/kernels
!make cuda
!ls -la ../../lib/

In [ ]:
# Install Ollama for comparison
!curl -fsSL https://ollama.com/install.sh | sh
!ollama --version

In [ ]:
# Start Ollama server in background
import subprocess
import time

# Start Ollama serve
ollama_process = subprocess.Popen(
    ['ollama', 'serve'],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)
time.sleep(5)
print('Ollama server started')

In [ ]:
# Pull a small model for testing
!ollama pull smollm:135m

## 2. Load CUDA Kernels

In [ ]:
import ctypes
import numpy as np
import time

# Load CUDA library
cuda_lib = ctypes.CDLL('../../lib/libtmac_kernel_cuda.so')

# Define function signatures
cuda_lib.cuda_available.restype = ctypes.c_int
cuda_lib.cuda_device_name.restype = ctypes.c_char_p
cuda_lib.cuda_init.argtypes = [ctypes.c_int, ctypes.c_int, ctypes.c_int]
cuda_lib.cuda_init.restype = ctypes.c_int
cuda_lib.cuda_cleanup.restype = None
cuda_lib.cuda_sync.restype = None

cuda_lib.tmac_matmul_cuda.argtypes = [
    ctypes.POINTER(ctypes.c_int8),   # weights
    ctypes.POINTER(ctypes.c_float),  # activations
    ctypes.POINTER(ctypes.c_float),  # output
    ctypes.POINTER(ctypes.c_float),  # scales
    ctypes.c_int, ctypes.c_int, ctypes.c_int  # M, N, K
]
cuda_lib.tmac_matmul_cuda.restype = ctypes.c_int

cuda_lib.rmsnorm_cuda.argtypes = [
    ctypes.POINTER(ctypes.c_float),
    ctypes.POINTER(ctypes.c_float),
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int, ctypes.c_int, ctypes.c_float
]
cuda_lib.rmsnorm_cuda.restype = ctypes.c_int

cuda_lib.softmax_cuda.argtypes = [
    ctypes.POINTER(ctypes.c_float),
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int, ctypes.c_int
]
cuda_lib.softmax_cuda.restype = ctypes.c_int

# Initialize CUDA
print(f'CUDA Device: {cuda_lib.cuda_device_name().decode()}')
cuda_lib.cuda_init(100_000_000, 10_000_000, 10_000_000)
print('CUDA initialized')

## 3. Benchmark: EdgeLLM CUDA Kernels

Simulate inference workload with realistic dimensions:
- SmolLM-135M: hidden_size=576, intermediate=1536, vocab=49152

In [ ]:
def benchmark_cuda_inference(batch_size=1, seq_len=1, hidden_size=576, 
                              intermediate_size=1536, vocab_size=49152,
                              num_layers=9, num_tokens=100):
    """
    Simulate transformer inference with CUDA kernels.
    
    Per token operations:
    - RMSNorm x 2 per layer
    - QKV projection (matmul)
    - Attention output projection
    - MLP up projection
    - MLP down projection
    - Final LM head
    """
    # Allocate buffers
    hidden = np.random.randn(batch_size * seq_len, hidden_size).astype(np.float32)
    weights_qkv = np.random.randint(-1, 2, (hidden_size * 3, hidden_size // 4), dtype=np.int8)
    weights_out = np.random.randint(-1, 2, (hidden_size, hidden_size // 4), dtype=np.int8)
    weights_up = np.random.randint(-1, 2, (intermediate_size, hidden_size // 4), dtype=np.int8)
    weights_down = np.random.randint(-1, 2, (hidden_size, intermediate_size // 4), dtype=np.int8)
    weights_lm = np.random.randint(-1, 2, (vocab_size, hidden_size // 4), dtype=np.int8)
    
    norm_weight = np.ones(hidden_size, dtype=np.float32)
    scales = np.ones(max(hidden_size * 3, intermediate_size, vocab_size), dtype=np.float32)
    
    # Output buffers
    output_qkv = np.zeros((batch_size * seq_len, hidden_size * 3), dtype=np.float32)
    output_out = np.zeros((batch_size * seq_len, hidden_size), dtype=np.float32)
    output_up = np.zeros((batch_size * seq_len, intermediate_size), dtype=np.float32)
    output_down = np.zeros((batch_size * seq_len, hidden_size), dtype=np.float32)
    output_lm = np.zeros((batch_size * seq_len, vocab_size), dtype=np.float32)
    norm_out = np.zeros_like(hidden)
    
    # Warmup
    for _ in range(10):
        cuda_lib.rmsnorm_cuda(
            norm_out.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
            hidden.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
            norm_weight.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
            batch_size * seq_len, hidden_size, ctypes.c_float(1e-6)
        )
    cuda_lib.cuda_sync()
    
    # Benchmark
    start = time.perf_counter()
    
    for token in range(num_tokens):
        for layer in range(num_layers):
            # Pre-attention RMSNorm
            cuda_lib.rmsnorm_cuda(
                norm_out.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
                hidden.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
                norm_weight.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
                batch_size * seq_len, hidden_size, ctypes.c_float(1e-6)
            )
            
            # QKV projection (simulated with matmul)
            cuda_lib.tmac_matmul_cuda(
                weights_qkv.ctypes.data_as(ctypes.POINTER(ctypes.c_int8)),
                norm_out.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
                output_qkv.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
                scales.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
                hidden_size * 3, batch_size * seq_len, hidden_size
            )
            
            # Attention output
            cuda_lib.tmac_matmul_cuda(
                weights_out.ctypes.data_as(ctypes.POINTER(ctypes.c_int8)),
                output_qkv.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
                output_out.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
                scales.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
                hidden_size, batch_size * seq_len, hidden_size * 3
            )
            
            # Post-attention RMSNorm
            cuda_lib.rmsnorm_cuda(
                norm_out.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
                output_out.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
                norm_weight.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
                batch_size * seq_len, hidden_size, ctypes.c_float(1e-6)
            )
            
            # MLP up
            cuda_lib.tmac_matmul_cuda(
                weights_up.ctypes.data_as(ctypes.POINTER(ctypes.c_int8)),
                norm_out.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
                output_up.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
                scales.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
                intermediate_size, batch_size * seq_len, hidden_size
            )
            
            # MLP down
            cuda_lib.tmac_matmul_cuda(
                weights_down.ctypes.data_as(ctypes.POINTER(ctypes.c_int8)),
                output_up.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
                output_down.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
                scales.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
                hidden_size, batch_size * seq_len, intermediate_size
            )
        
        # Final LM head
        cuda_lib.rmsnorm_cuda(
            norm_out.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
            output_down.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
            norm_weight.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
            batch_size * seq_len, hidden_size, ctypes.c_float(1e-6)
        )
        
        cuda_lib.tmac_matmul_cuda(
            weights_lm.ctypes.data_as(ctypes.POINTER(ctypes.c_int8)),
            norm_out.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
            output_lm.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
            scales.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
            vocab_size, batch_size * seq_len, hidden_size
        )
        
        # Softmax for sampling
        cuda_lib.softmax_cuda(
            output_lm.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
            output_lm.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
            batch_size * seq_len, vocab_size
        )
    
    cuda_lib.cuda_sync()
    end = time.perf_counter()
    
    elapsed = end - start
    tok_per_sec = num_tokens / elapsed
    
    return {
        'tokens': num_tokens,
        'elapsed_sec': elapsed,
        'tok_per_sec': tok_per_sec,
        'ms_per_token': (elapsed / num_tokens) * 1000
    }

In [ ]:
# Run EdgeLLM CUDA benchmark
print('EdgeLLM CUDA Benchmark (SmolLM-135M simulation)')
print('=' * 50)

cuda_results = benchmark_cuda_inference(num_tokens=100)

print(f"Tokens generated: {cuda_results['tokens']}")
print(f"Total time: {cuda_results['elapsed_sec']:.3f}s")
print(f"Throughput: {cuda_results['tok_per_sec']:.1f} tok/s")
print(f"Latency: {cuda_results['ms_per_token']:.2f} ms/token")

## 4. Benchmark: Ollama

In [ ]:
import requests
import json

def benchmark_ollama(model='smollm:135m', prompt='Write a story about', num_tokens=100):
    """
    Benchmark Ollama inference.
    """
    url = 'http://localhost:11434/api/generate'
    
    payload = {
        'model': model,
        'prompt': prompt,
        'stream': False,
        'options': {
            'num_predict': num_tokens,
            'temperature': 0.7
        }
    }
    
    start = time.perf_counter()
    response = requests.post(url, json=payload)
    end = time.perf_counter()
    
    if response.status_code != 200:
        print(f'Error: {response.status_code}')
        print(response.text)
        return None
    
    data = response.json()
    
    # Extract metrics from Ollama response
    eval_count = data.get('eval_count', num_tokens)
    eval_duration = data.get('eval_duration', 0) / 1e9  # nanoseconds to seconds
    
    if eval_duration > 0:
        tok_per_sec = eval_count / eval_duration
    else:
        elapsed = end - start
        tok_per_sec = eval_count / elapsed
        eval_duration = elapsed
    
    return {
        'tokens': eval_count,
        'elapsed_sec': eval_duration,
        'tok_per_sec': tok_per_sec,
        'ms_per_token': (eval_duration / eval_count) * 1000 if eval_count > 0 else 0,
        'response': data.get('response', '')[:100]
    }

In [ ]:
# Run Ollama benchmark
print('Ollama Benchmark (SmolLM-135M)')
print('=' * 50)

ollama_results = benchmark_ollama(num_tokens=100)

if ollama_results:
    print(f"Tokens generated: {ollama_results['tokens']}")
    print(f"Total time: {ollama_results['elapsed_sec']:.3f}s")
    print(f"Throughput: {ollama_results['tok_per_sec']:.1f} tok/s")
    print(f"Latency: {ollama_results['ms_per_token']:.2f} ms/token")
    print(f"\nSample output: {ollama_results['response']}...")

## 5. Head-to-Head Comparison

In [ ]:
print('\n' + '=' * 60)
print('BENCHMARK COMPARISON: EdgeLLM CUDA vs Ollama')
print('=' * 60)
print(f"{'Metric':<25} {'EdgeLLM CUDA':>15} {'Ollama':>15} {'Winner':>10}")
print('-' * 60)

if ollama_results:
    # Throughput
    cuda_tps = cuda_results['tok_per_sec']
    ollama_tps = ollama_results['tok_per_sec']
    winner_tps = 'EdgeLLM' if cuda_tps > ollama_tps else 'Ollama'
    print(f"{'Throughput (tok/s)':<25} {cuda_tps:>15.1f} {ollama_tps:>15.1f} {winner_tps:>10}")
    
    # Latency
    cuda_lat = cuda_results['ms_per_token']
    ollama_lat = ollama_results['ms_per_token']
    winner_lat = 'EdgeLLM' if cuda_lat < ollama_lat else 'Ollama'
    print(f"{'Latency (ms/token)':<25} {cuda_lat:>15.2f} {ollama_lat:>15.2f} {winner_lat:>10}")
    
    # Speedup
    speedup = cuda_tps / ollama_tps if ollama_tps > 0 else 0
    print('-' * 60)
    print(f"{'Speedup':<25} {speedup:>15.2f}x")
else:
    print('Ollama benchmark failed - showing EdgeLLM results only')
    print(f"{'Throughput (tok/s)':<25} {cuda_results['tok_per_sec']:>15.1f}")
    print(f"{'Latency (ms/token)':<25} {cuda_results['ms_per_token']:>15.2f}")

## 6. Extended Benchmark (Multiple Runs)

In [ ]:
# Run multiple iterations for statistical significance
print('Running extended benchmark (5 iterations)...')
print()

cuda_runs = []
ollama_runs = []

for i in range(5):
    print(f'Run {i+1}/5...')
    
    # EdgeLLM CUDA
    r = benchmark_cuda_inference(num_tokens=50)
    cuda_runs.append(r['tok_per_sec'])
    
    # Ollama
    r = benchmark_ollama(num_tokens=50)
    if r:
        ollama_runs.append(r['tok_per_sec'])

print()
print('Extended Results:')
print(f"EdgeLLM CUDA: {np.mean(cuda_runs):.1f} ± {np.std(cuda_runs):.1f} tok/s")
if ollama_runs:
    print(f"Ollama:       {np.mean(ollama_runs):.1f} ± {np.std(ollama_runs):.1f} tok/s")
    print(f"\nSpeedup: {np.mean(cuda_runs) / np.mean(ollama_runs):.2f}x")

In [ ]:
# Cleanup
cuda_lib.cuda_cleanup()
ollama_process.terminate()
print('Cleanup complete')

## Summary

### Key Findings:

1. **EdgeLLM CUDA** uses T-MAC (table lookup) instead of multiply-accumulate
2. **BitNet 1.58-bit** quantization provides 6.5x compression
3. **CUDA kernels** leverage shared memory for fast LUT access

### Why EdgeLLM Can Be Faster:

- **No FP16/FP32 multiplications** - just table lookups
- **Smaller model size** - fits in L2 cache
- **Lower memory bandwidth** - main bottleneck for LLMs
- **Deterministic latency** - no GC pauses (Mojo)

### Target Performance:

| Hardware | EdgeLLM Target | Ollama Typical |
|----------|----------------|----------------|
| Tesla T4 | 200-400 tok/s | 100-150 tok/s |
| RTX 3090 | 400-600 tok/s | 150-200 tok/s |
| Jetson Nano | 80-120 tok/s | 20-40 tok/s |